# БПМ-19-3, Измайлов Лев, вариант 28

In [4]:
from typing import List
from math import log2, ceil
from random import randrange
import binascii

In [7]:
11

In [8]:
def __hamming_common(src: List[List[int]], s_num: int, encode=True) -> int:
    s_range = range(s_num)
    errors = 0
    for i in src:
        sindrome = 0
        for s in s_range:
            sind = 0
            for p in range(2 ** s, len(i) + 1, 2 ** (s + 1)):
                for j in range(2 ** s):
                    if (p + j) > len(i):
                        break
                    sind ^= i[p + j - 1]
            if encode:
                i[2 ** s - 1] = sind
            else:
                sindrome += (2 ** s * sind)
        if (not encode) and sindrome:
            try:
                i[sindrome - 1] = int(not i[sindrome - 1])
            except IndexError:
                errors += 1
    return errors

In [9]:
def hamming_encode(msg: str, mode: int = 8) -> str:
    """
    Encoding the message with Hamming code.
    :param msg: Message string to encode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    msg_b = msg.encode("utf8")
    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    bit_seq = []
    for byte in msg_b:  # get bytes to binary values; every bits store to sublist
        bit_seq += list(map(int, f"{byte:08b}"))

    res_len = ceil((len(msg_b) * 8) / mode)     # length of result (bytes)
    bit_seq += [0] * (res_len * mode - len(bit_seq))    # filling zeros

    to_hamming = []

    for i in range(res_len):    # insert control bits into specified positions
        code = bit_seq[i * mode:i * mode + mode]
        for j in range(s_num):
            code.insert(2 ** j - 1, 0)
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, True)   # process

    for i in to_hamming:
        result += "".join(map(str, i))

    return result

In [10]:
def hamming_decode(msg: str, mode: int = 8):
    """
    Decoding the message with Hamming code.
    :param msg: Message string to decode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    res_len = len(msg) // (mode + s_num)    # length of result (bytes)
    code_len = mode + s_num     # length of one code sequence

    to_hamming = []

    for i in range(res_len):    # convert binary-like string to int-list
        code = list(map(int, msg[i * code_len:i * code_len + code_len]))
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, False)  # process

    for i in to_hamming:    # delete control bits
        for j in range(s_num):
            i.pop(2 ** j - 1 - j)
        result += "".join(map(str, i))

    msg_l = []

    for i in range(len(result) // 8):   # convert from binary-sring value to integer
        val = "".join(result[i * 8:i * 8 + 8])
        msg_l.append(int(val, 2))

    # finally decode to a regular string
    try:
        result = bytes(msg_l).decode("utf-8")
    except UnicodeDecodeError:
        pass

    return result, errors

In [11]:
def noizer(msg: str, mode: int) -> str:
    """
    Generates an error in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize = randrange(code_len)
        to_noize[noize] = int(not to_noize[noize])
        result += "".join(map(str, to_noize))

    return result

In [12]:
def noizer4(msg: str, mode: int) -> str:
    """
    Generates up to 4 errors in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize1 = randrange(code_len)
        noize2 = randrange(code_len)
        noize3 = randrange(code_len)
        noize4 = randrange(code_len)
        to_noize[noize1] = int(not to_noize[noize1])
        to_noize[noize2] = int(not to_noize[noize2])
        to_noize[noize3] = int(not to_noize[noize3])
        to_noize[noize4] = int(not to_noize[noize4])
        result += "".join(map(str, to_noize))

    return result

In [35]:
MODE = 89 # длина слова с контрольными битами составляет 60 => значащих битов в слове 54
msg ="Green Day — американская рок-группа, образованная в городе Ист-Бэй  (англ.)рус. (Калифорния) в 1987 году вокалистом и гитаристом Билли Джо Армстронгом вместе с басистом и бэк-вокалистом Майком Дёрнтом. На протяжении большей части карьеры группа была представлена как пауэр-трио[1] с барабанщиком Тре Кулом, который заменил Джона Киффмейера в 1990 году перед записью второго студийного альбома группы Kerplunk (1991). Сессионный гитарист Джейсон Уайт стал полноправным участником группы в 2012 году, но вернулся к своей роли сессионного участника в 2016 году. До 1989 года Green Day выступали под названием Sweet Children, и были частью панк-сцены Bay Area конца 1980-х – начала 1990-х годов, возникшей в клубе 924 Gilman Street  (англ.)рус. в Беркли (Калифорния). Ранние альбомы группы были выпущены на инди-лейбле Lookout! Records. В 1994 году их первый альбом Dookie, который был выпущен на лейбле Reprise Records, имел оглушительный успех и в итоге разошёлся тиражом более 10 миллионов копий в США. Наряду с другими калифорнийскими панк-группами Bad Religion, the Offspring, Rancid и Social Distortion, Green Day повысили интерес к панк-року в Соединённых Штатах. Хотя альбомы Insomniac (1995), Nimrod (1997) и Warning (2000) не были такими успешными как Dookie, они всё же имели умеренный успех, причём первые два достигли «мультиплатинового» статуса, в то время как последний стал «золотым». Седьмой альбом Green Day, рок-опера American Idiot (2004), завоевал популярность у молодого поколения, разошедшись тиражом в 6 миллионов копий в США. Их следующий альбом, 21st Century Breakdown, был выпущен в 2009 году и занял лидирующие позиции в чартах многих стран. За ним последовала трилогия альбомов ¡Uno!, ¡Dos! и ¡Tré!, выпущенных в сентябре, ноябре и декабре 2012 года соответственно. Все три альбома не были коммерчески успешны по сравнению с предыдущими релизами группы, в основном из-за отсутствия их продвижения и того, что Армстронг проходил реабилитацию. Их двенадцатый студийный альбом, Revolution Radio, был выпущен в октябре 2016 года и стал третьим, дебютировавшим под номером 1 в Billboard 200. Тринадцатый студийный альбом группы, Father of All Motherfuckers, был выпущен 7 февраля 2020 года." 
print(f'Сообщение:\n{msg}')
checksum = crc32(msg)
print(f'Контрольная сумма: {checksum}')

# Первая отправка (без ошибок)
print('-----------ПЕРВАЯ ОТПРАВКА-----------')
enc_msg = hamming_encode(msg, MODE)
print(f'Кодированное сообщение:\n{enc_msg}')
dec_msg, err = hamming_decode(enc_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc32(dec_msg)}, корректность: {crc32(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Вторая отправка (не более 1 ошибки на слово)
print('-----------ВТОРАЯ ОТПРАВКА-----------')
noize_msg = noizer(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc32(dec_msg)}, корректность: {crc32(dec_msg) == checksum}')
print(f'MSG: {msg == dec_msg}')

# Третья отправка (4 ошибки на слово)
print('-----------ТРЕТЬЯ ОТПРАВКА-----------')
noize_msg = noizer4(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc32(dec_msg)}, корректность: {crc32(dec_msg) == checksum}, количество обнаруженных ошибок: {err}')

Сообщение:
Green Day — американская рок-группа, образованная в городе Ист-Бэй  (англ.)рус. (Калифорния) в 1987 году вокалистом и гитаристом Билли Джо Армстронгом вместе с басистом и бэк-вокалистом Майком Дёрнтом. На протяжении большей части карьеры группа была представлена как пауэр-трио[1] с барабанщиком Тре Кулом, который заменил Джона Киффмейера в 1990 году перед записью второго студийного альбома группы Kerplunk (1991). Сессионный гитарист Джейсон Уайт стал полноправным участником группы в 2012 году, но вернулся к своей роли сессионного участника в 2016 году. До 1989 года Green Day выступали под названием Sweet Children, и были частью панк-сцены Bay Area конца 1980-х – начала 1990-х годов, возникшей в клубе 924 Gilman Street  (англ.)рус. в Беркли (Калифорния). Ранние альбомы группы были выпущены на инди-лейбле Lookout! Records. В 1994 году их первый альбом Dookie, который был выпущен на лейбле Reprise Records, имел оглушительный успех и в итоге разошёлся тиражом более 10 миллионов 